<a href="https://colab.research.google.com/github/Yolantele/ML-data-clasifier/blob/master/tabular_neural_net_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular Model For Waste Data Classification


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.
df - data_frame

In [0]:
# mount to google drive and load files
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# check file is loaded OK
path = '/content/drive/My Drive/data/'
print(path)

/content/drive/My Drive/data/


In [0]:
# load train data frame
df = pd.read_csv(path + '/minimizedData.csv')



# load test data frame with material field empty
test_df = pd.read_csv(path + '/testNoMaterialData.csv')

df.head()

,reason,origin,color,state,size,consistency,otherCode,material4,material3,material2,material,mType,composite2,composite1,cType,indirectProduct,directProduct,pType,mixedOrPure,cleanOrDirty,euralDescription,euralCode,description,/0
0,NaN,NaN,NaN,NaN,NaN,slurry,NaN,NaN,NaN,oil,organic material,Soy,NaN,dry,NaN,dry,NaN,NaN,1.0,NaN,material unsuitable for consumption or processing,20304,Soyadroes technically,NaN
1,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,wood,branches,NaN,NaN,NaN,wood,NaN,NaN,0.0,NaN,waste from forestry,20107,Branches,NaN
2,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,cocoa,caps,NaN,NaN,NaN,cocoa,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,Cocoa shells,NaN
3,NaN,NaN,NaN,debris,NaN,vast,NaN,NaN,NaN,NaN,sand,NaN,NaN,debris,NaN,debris,NaN,NaN,1.0,1.0,"mixtures of concrete, stones, tiles or ceramic...",170107,Debris with Sand,NaN
4,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,organic material,Soy,NaN,NaN,NaN,organic material,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,soya,NaN


#cats and conts 

**categorical variables - cat_names:**

what you're using to make predictions with.  (description, euralDescription, composite1, mType).
for categorical variables - will use Embeddings 

**continuos variables - cont_names:**

any catgeory that has limited number of possible (integer) choises: true/false ; fields such as cleanOrContaminated or mixedOrPure
continuos varibale names - they are sent to Neural net as pixels 

**uses processor - procs:**

a number of processes that deal with data in some ways: FillMissing, Categorify, Normalize both train and test dtata sets

In [0]:
# dependable variable field:
dep_var = 'material'
 
# can add composite1, mType
cat_names = ['euralDescription', 'description', 'euralCode']
cont_names = []
# cont_names = ['mixedOrPure', 'cleanOrDirty']
# processors:
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[950:1200].copy(), path=path + '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names)

Next:
data block api
add labels - tell which cols - column it is 

and get back data bunch
split train and test data ranges 800,1000


In [0]:
data = (TabularList.from_df(df, path=path+ '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(950,1200)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:537: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
PCFIC, non-iron, solvents, syrup, flint...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [0]:
data.show_batch(rows=10)

euralDescription,description,euralCode,target
paper and cardboard,Envelopes,200101,paper
wood,"B-Wood, painted, glued sheet material",170201,wood
metal,Demolition,200140,metal
building material containing asbestos,"asbestos, not route",170605,asbestos
#na#,2330 b wood,170904,wood
metal waste,Iron,20110,iron
biodegradable waste,Gardening system heavy / chips,200201,organic material
soil and stones not covered by 17 05 03,category 1 soil,170504,ground
plastics,PA / PE - foil,200139,PA
biodegradable waste,Organic waste (NTA 610),200201,organic material


pass in data to tabular learner:


In [0]:
learn = tabular_learner(data, layers=[300,100], metrics=accuracy)

In [0]:
learn.fit(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.900922,2.334291,0.565574,00:00
1,1.188990,1.174783,0.717213,00:00


In [73]:
# see into the model
# learn.model.eval()
learn.summary()


TabularModel
Layer (type)         Output Shape         Param #    Trainable 
Embedding            [15]                 855        True      
______________________________________________________________________
Embedding            [104]                179,088    True      
______________________________________________________________________
Embedding            [18]                 1,404      True      
______________________________________________________________________
Dropout              [137]                0          False     
______________________________________________________________________
Linear               [300]                41,400     True      
______________________________________________________________________
ReLU                 [300]                0          False     
______________________________________________________________________
BatchNorm1d          [300]                600        True      
_________________________________________________

In [0]:
learn.model_dir = '/content/drive/My Drive/data/models'
learn.save('material_classifier_model')

## Inference

data_frame (df) 

In [97]:
row = 995
# data_frame = df
data_frame = test_df

print('given material -->', data_frame.iloc[row].material)
print(data_frame.iloc[row].description, ', ', data_frame.iloc[row].description,)


given material --> nan
alcoholic beverages ,  alcoholic beverages


In [98]:
# train data prediction
# one_row = df.iloc[row]

# test data prediction
one_row = test_df.iloc[row]



predicted_category = learn.predict(one_row)
total_categories = learn.predict(one_row)[1]
prediction_weights = learn.predict(one_row)[2]

print('predicted material ------> ', predicted_category)
print('')
print('total available material categories:', total_categories)

conf_r = max(learn.predict(one_row)[2])
print('confidense rate ---------> ', conf_r)
# print('prediction weights to categories:'prediction_weights)


predicted material ------>  (Category water, tensor(91), tensor([0.0013, 0.0011, 0.0042, 0.0070, 0.0017, 0.0031, 0.0028, 0.0018, 0.0023,
        0.0048, 0.0012, 0.0017, 0.0028, 0.0020, 0.0045, 0.0150, 0.0039, 0.0013,
        0.0027, 0.0022, 0.0079, 0.0022, 0.0053, 0.0014, 0.0274, 0.0099, 0.0012,
        0.0618, 0.0022, 0.0062, 0.0044, 0.0037, 0.0088, 0.0607, 0.0023, 0.0021,
        0.0045, 0.0191, 0.0077, 0.0056, 0.0017, 0.0070, 0.0016, 0.0150, 0.0030,
        0.0027, 0.0049, 0.0084, 0.0024, 0.0036, 0.0049, 0.0156, 0.0863, 0.0019,
        0.0027, 0.0027, 0.0023, 0.0029, 0.0007, 0.0049, 0.0015, 0.0061, 0.0015,
        0.0049, 0.0029, 0.0024, 0.0021, 0.0783, 0.0044, 0.0468, 0.0107, 0.0073,
        0.0026, 0.0012, 0.0015, 0.0042, 0.0032, 0.0021, 0.0024, 0.0150, 0.0052,
        0.0025, 0.0220, 0.0183, 0.0019, 0.0036, 0.0168, 0.0035, 0.0061, 0.0013,
        0.0020, 0.2171, 0.0083, 0.0019, 0.0012]))

total available material categories: tensor(91)
confidense rate --------->  tensor(0.2171)



- tfm - transformations,
- train_ds - trained data set,
- df_tst = data fram etest,


In [0]:
# apply the set of transforms to your new data:
for tfm in data.tfms: df = tfm(df, is_test=True)

# Then you have to transforms the categorical variables into codes:
cats = np.stack([c.cat.codes.values for n,c in df[cat_names].items()], 1) + 1

# and normalize the continuous variables
conts = np.stack([c.astype('float32').values for n,c in df[cont_names].items()], 1)
means, stds = data.stats 
conts = (conts - means[None]) / (stds[None]+1e-7)

# And finally, you can run the model on [tensor(cats),tensor(conts)] (or some batches of it if you have a lot things to predict).